In [2]:
import numpy as np
import torch
import random
import matplotlib.pyplot as plt

from torch import nn, optim
from collections import Counter

In [4]:
a = np.random.rand(3)
b = np.random.rand(4)

np.dot(a.reshape(-1,1), b.reshape(1, -1))

array([[0.0309803 , 0.01987633, 0.00189091, 0.00353473],
       [0.05293239, 0.03396035, 0.00323077, 0.00603937],
       [0.75037573, 0.48142586, 0.04579982, 0.08561482]])

In [2]:
text = "I like dog i like cat i like animal dog cat animal apple cat dog like dog fish milk like dog \
cat eyes like i like apple apple i hate apple i movie book music like cat dog hate cat dog like"

In [3]:
EMBEDDING_DIM = 2
PRINT_EVERY = 1000
EPOCHS = 1000
BATCH_SIZE = 5
N_SAMPLES = 3
WINDOW_SIZE = 5
FREQ = 0
DELETE_WORDS = False

In [4]:
def preprocess(text, FREQ):
    text = text.lower()
    words = text.split()
    
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > FREQ]
    
    return trimmed_words

words = preprocess(text, FREQ)

In [10]:
vocab = set(words)
word2id = { word: idx for idx, word in enumerate(vocab) }
id2word = { idx: word for idx, word in enumerate(vocab) }

In [12]:
int_words = [word2id[word] for word in words]

In [17]:
int_words_counts = Counter(int_words)
total_count = len(int_words)
word_freqs = {w: c/total_count for w, c in int_words_counts.items()}

In [18]:
word_freqs = np.array(list(word_freqs.values()))
unigram_dist = word_freqs / word_freqs.sum()
noise_dist = torch.from_numpy(unigram_dist ** (0.75) / np.sum(unigram_dist ** (0.75)))

In [23]:
def get_target(words, idx, WINDOW_SIZE):
    target_window = np.random.randint(1, WINDOW_SIZE + 1)
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    targets = set( words[start_point:idx] + words[idx + 1:end_point + 1] )
    
    return list(targets)

def get_batch(words, BATCH_SIZE, WINDOW_SIZE):
    n_batches = len(words) // BATCH_SIZE
    words = words[:n_batches * BATCH_SIZE]
        
    for idx in range(0, len(words), BATCH_SIZE):
        batch_x, batcy_y = [], []
        batch = words[idx:idx + BATCH_SIZE]
        
        for i in range(len(batch)):
            x = batch[i]
            y = get_batch(batch, i, WINDOW_SIZE)
            batch_x.extend([x] * len(y))
            batch_y.extend(y)
            
        yield batch_x, batch_y

In [ ]:
class SkipGramNeg(nn.Module):
    
    def __init__(self, n_vocab, n_embed, noise_dist):
        super(self, SkipGramNeg).__init__()
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)
        
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)
        
    def forward_input(self, input_words):
        inc_vec = self.in_embed(input_words)
        return inc_vec
    
    def 